In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/media/minhduc0711/Libraries/Codes/EURECOM/MALIS_project


In [4]:
import torchaudio
from torchaudio import transforms

from src.models.onsets_and_frames import OnsetsAndFrames
from src.data.data_modules import MAPSDataModule

In [5]:
model = OnsetsAndFrames.load_from_checkpoint("models/onf-MAPS-epoch=379-valid_loss=0.08.ckpt",
                                             in_feats=229)
model.eval();

In [6]:
audio_path = "canon1.wav"

audio, sample_rate = torchaudio.load(str(audio_path))
if sample_rate != 16000:
    resampler = transforms.Resample(orig_freq=sample_rate,
                                   new_freq=16000)
    audio = resampler(audio)
# convert to mono channel if necessary
if audio.ndim > 1:
    audio = audio.mean(dim=0)

dm = MAPSDataModule(batch_size=1)
audio = dm.audio_transform(audio)
audio.transpose_(0, 1)
audio.unsqueeze_(0)

audio.shape

torch.Size([1, 336, 229])

In [7]:
onset_pred, frame_pred, velocity_pred = model(audio)
p_est, i_est, v_est = model.extract_notes(onset_pred.squeeze(),
                                          frame_pred.squeeze(),
                                          velocity_pred.squeeze(),
                                          sample_rate=sample_rate,
                                          hop_length=512)

In [14]:
from mido import MidiFile, MidiTrack, Message
from mir_eval.util import hz_to_midi
from pathlib import Path

file = MidiFile()
track = MidiTrack()
file.tracks.append(track)
ticks_per_second = file.ticks_per_beat * 5

events = []
for pitch, (time_on, time_off), velocity in \
        zip(p_est, i_est, v_est):
    events.append({"type":"on", "pitch":pitch, "time":time_on, "velocity":velocity})
    events.append({"type":"off", "pitch":pitch, "time":time_off, "velocity":velocity})
events.sort(key=lambda row: row["time"])

last_tick = 0
for event in events:
    current_tick = int(event['time'] * ticks_per_second)
    velocity = min(int(event['velocity'] * 127), 127)
    pitch = int(round(hz_to_midi(event['pitch'])))
    track.append(Message('note_' + event['type'], note=pitch, velocity=velocity, time=current_tick - last_tick))
    last_tick = current_tick

file.save(Path(audio_path).with_suffix(".midi"))

In [17]:
from src.data.data_modules import MAPSDataModule
dm = MAPSDataModule(batch_size=1)
dm.setup("test")

Loading data samples into memory: 100%|██████████| 60/60 [00:42<00:00,  1.41it/s]


In [27]:
loader =dm.test_dataloader()

next(iter(loader))["audio"].shape

torch.Size([1, 12045, 229])

In [24]:
dm.test_ds[2]["audio"].shape

torch.Size([8568, 229])